## Solver CFT Program

In [60]:
# -*- coding: utf-8 -*-
# Kernel: Python 3.9.12

# Install OR-Tools for Python
# !pip install ortools
# !pip3 install --upgrade --user ortools

# https://developers.google.com/optimization/examples

### Import the required libraries.

In [48]:
%pip install ortools
!pip3 install --upgrade --user ortools

Note: you may need to restart the kernel to use updated packages.


In [49]:
# from absl import app
# from absl import flags

from google.protobuf import text_format
from ortools.sat.python import cp_model


import pandas as pd
import numpy as np

### Declare the Demand

In [50]:
# Numero de atenciones por dia/semana/mes a optimizar
# demand = [14, 13, 15, 16, 19, 18, 11]


### Declare the solver.


In [51]:
model = cp_model.CpModel()

### Create the variables.

In [52]:
num_prof = 4
num_shifts = 3
num_days = 3
all_prof = range(num_prof)
all_shifts = range(num_shifts)
all_days = range(num_days)


# La matriz define las asignaciones para los turnos a las enfermeras de la siguiente manera:
# shifts [(n, d, s)] es igual a 1 si el turno s se asigna a la enfermera n el día d, y 0 de lo contrario.

shifts = {}
for n in all_prof:
    for d in all_days:
        for s in all_shifts:
            shifts[(n, d, s)] = model.NewBoolVar('shift_n%id%is%i' % (n, d, s))

### Define the constraints.

#### Asignación de atenciónes
* Cada turno se asigna a una sola enfermera al día.
* Cada enfermera trabaja como máximo un turno al día.
* Este es el código que crea la primera condición.

In [53]:
for d in all_days:
    for s in all_shifts:
        model.AddExactlyOne(shifts[(n, d, s)] for n in all_prof)

# La última línea dice que para cada turno, la suma de los profesionales asignados a esas atenciones es 1.


# A continuación, aquí está el código que requiere que cada profesional trabaje como máximo un turno al día

for n in all_prof:
    for d in all_days:
        model.AddAtMostOne(shifts[(n, d, s)] for s in all_shifts)

# Para cada profesional, la suma de los turnos asignados a ese profesional es como máximo 1 ("a lo sumo" porque un profesional podría tener el día libre)

### Asignar atenciones de manera uniforme
A continuación, mostramos cómo asignar turnos a los profesionales de manera más uniforme posible:

 * Dado que hay (X) atenciones durante el período de (X) días, podemos asignar dos **atenciones** a cada una de los **(X) profesionales** .
 * Después de eso, quedará una atención, que se puede asignar a cualquier profesional.

El siguiente código garantiza que cada profesional trabaje al menos dos atenciones en el período de (X) días.

In [54]:
# Try to distribute the shifts evenly, so that each nurse works
# min_shifts_per_nurse shifts. If this is not possible, because the total
# number of shifts is not divisible by the number of nurses, some nurses will
# be assigned one more shift.
min_shifts_per_prof = (num_shifts * num_days) // num_prof
if num_shifts * num_days % num_prof == 0:
    max_shifts_per_prof = min_shifts_per_prof
else:
    max_shifts_per_prof = min_shifts_per_prof + 1
for n in all_prof:
    num_shifts_worked = []
    for d in all_days:
        for s in all_shifts:
            num_shifts_worked.append(shifts[(n, d, s)])
    model.Add(min_shifts_per_prof <= sum(num_shifts_worked))
    model.Add(sum(num_shifts_worked) <= max_shifts_per_prof)

In [55]:
# so you can assign at least two shifts to each prof. This is guaranteed by the constraint
model.Add(min_shifts_per_prof <= sum(num_shifts_worked))

# Dado que hay nueve turnos en total durante el período de tres días, queda un turno después de asignar dos turnos a cada enfermera.
# El turno extra se puede asignar a cualquier enfermera.
# La última línea garantiza que a ningun profesional se le asigne más de un turno adicional.
# La restricción no es necesaria en este caso, porque solo hay un turno adicional.
# Pero para diferentes valores de parámetros, podría haber varios cambios adicionales, en cuyo caso la restricción es necesaria
model.Add(sum(num_shifts_worked) <= max_shifts_per_prof)

### Define the objective function.

### Invoke the solver and display the results.

#### Actualizar los parámetros del solucionador
 * En un modelo de no optimización, puedes habilitar la búsqueda de todas las soluciones.

In [56]:
solver = cp_model.CpSolver()
solver.parameters.linearization_level = 0
# Enumerate all solutions.
solver.parameters.enumerate_all_solutions = True

### Register a Solutions Callback

In [57]:

class ProfPartialSolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, shifts, num_prof, num_days, num_shifts, limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self._shifts = shifts
        self._num_prof = num_prof
        self._num_days = num_days
        self._num_shifts = num_shifts
        self._solution_count = 0
        self._solution_limit = limit

    def on_solution_callback(self):
        self._solution_count += 1
        print('Solution %i' % self._solution_count)
        for d in range(self._num_days):
            print('Day %i' % d)
            for n in range(self._num_prof):
                is_working = False
                for s in range(self._num_shifts):
                    if self.Value(self._shifts[(n, d, s)]):
                        is_working = True
                        print('  Profesional %i works shift %i' % (n, s))
                if not is_working:
                    print('  Profesional {} does not work'.format(n))
        if self._solution_count >= self._solution_limit:
            print('Stop search after %i solutions' % self._solution_limit)
            self.StopSearch()

    def solution_count(self):
        return self._solution_count

# Display the first five solutions.
solution_limit = 5
solution_printer = ProfPartialSolutionPrinter(shifts, num_prof,
                                                num_days, num_shifts,
                                                solution_limit)

### Invoke the Solver

In [58]:
solver.Solve(model, solution_printer)

Solution 1
Day 0
  Profesional 0 does not work
  Profesional 1 works shift 0
  Profesional 2 works shift 1
  Profesional 3 works shift 2
Day 1
  Profesional 0 works shift 2
  Profesional 1 does not work
  Profesional 2 works shift 1
  Profesional 3 works shift 0
Day 2
  Profesional 0 works shift 2
  Profesional 1 works shift 1
  Profesional 2 works shift 0
  Profesional 3 does not work
Solution 2
Day 0
  Profesional 0 works shift 0
  Profesional 1 does not work
  Profesional 2 works shift 1
  Profesional 3 works shift 2
Day 1
  Profesional 0 does not work
  Profesional 1 works shift 2
  Profesional 2 works shift 1
  Profesional 3 works shift 0
Day 2
  Profesional 0 works shift 2
  Profesional 1 works shift 1
  Profesional 2 works shift 0
  Profesional 3 does not work
Solution 3
Day 0
  Profesional 0 works shift 0
  Profesional 1 does not work
  Profesional 2 works shift 1
  Profesional 3 works shift 2
Day 1
  Profesional 0 works shift 1
  Profesional 1 works shift 2
  Profesional 2 doe

2

## Soluciones

 * Statistics

In [59]:
# Statistics.
print('\nStatistics')
print('  - conflicts      : %i' % solver.NumConflicts())
print('  - branches       : %i' % solver.NumBranches())
print('  - wall time      : %f s' % solver.WallTime())
print('  - solutions found: %i' % solution_printer.solution_count())


#if __name__ == '__main__': main()


Statistics
  - conflicts      : 5
  - branches       : 142
  - wall time      : 0.002153 s
  - solutions found: 5
